In [16]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [35]:
attr = {      'StandardStats': 'stats_standard_11',
              'Goalkeeping': 'stats_keeper_adv_11',
              'Shooting': 'stats_shooting_11',
              'Passing': 'stats_passing_11',
              'Pass_types': 'stats_passing_types_11',
              'GCA': 'stats_gca_11',
              'Defense': 'stats_defense_11',
              'Possession': 'stats_possession_11',
              'Playing_time': 'stats_playing_time_11',
              'Misc': 'stats_misc_11'
             }

In [18]:
Leagues = { 'Serie A': ['https://fbref.com/en/comps/11/2022-2023/2022-2023-Serie-A-Stats'],
            'Premier League': ['https://fbref.com/en/comps/9/2022-2023/2022-2023-Premier-League-Stats'],
            'La Liga': ['https://fbref.com/en/comps/12/2022-2023/2022-2023-La-Liga-Stats'],
            'Bundesliga': ['https://fbref.com/en/comps/20/2022-2023/2022-2023-Bundesliga-Stats'],
            'Ligue 1': ['https://fbref.com/en/comps/13/2022-2023/2022-2023-Ligue-1-Stats']}

In [21]:
url = "https://fbref.com/en/comps/11/2022-2023/2022-2023-Serie-A-Stats"

response = requests.get(url)
SerieA = {}
# Check if the request was successful
if response.status_code == 200:
    # Parse the HTML content
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Select the table containing player statistics
    table = soup.select('#results2022-2023111_overall tbody')
    # Check if the table is found
    if table:
        # Convert the table to string and create a new BeautifulSoup object
        table_html = str(table[0])
        table_soup = BeautifulSoup(table_html, 'html.parser')
        
        # Find all table rows within the table
        clubs = table_soup.find_all('tr')
        
        # Iterate over each row and extract player information
        for row in clubs:
            # Extract player name
            Club = row.find('td', {'data-stat': 'team'}).text.strip()

            link = 'https://fbref.com/' + row.find('a')['href']

            # Print or store the extracted information
            SerieA[Club] = link
    else:
        print("Table not found on the page.")
else:
    print("Failed to retrieve data. Status code:", response.status_code)
print(SerieA)

{'Napoli': 'https://fbref.com//en/squads/d48ad4ff/2022-2023/Napoli-Stats', 'Lazio': 'https://fbref.com//en/squads/7213da33/2022-2023/Lazio-Stats', 'Inter': 'https://fbref.com//en/squads/d609edc0/2022-2023/Internazionale-Stats', 'Milan': 'https://fbref.com//en/squads/dc56fe14/2022-2023/Milan-Stats', 'Atalanta': 'https://fbref.com//en/squads/922493f3/2022-2023/Atalanta-Stats', 'Roma': 'https://fbref.com//en/squads/cf74a709/2022-2023/Roma-Stats', 'Juventus': 'https://fbref.com//en/squads/e0652b02/2022-2023/Juventus-Stats', 'Fiorentina': 'https://fbref.com//en/squads/421387cf/2022-2023/Fiorentina-Stats', 'Bologna': 'https://fbref.com//en/squads/1d8099f8/2022-2023/Bologna-Stats', 'Torino': 'https://fbref.com//en/squads/105360fe/2022-2023/Torino-Stats', 'Monza': 'https://fbref.com//en/squads/21680aa4/2022-2023/Monza-Stats', 'Udinese': 'https://fbref.com//en/squads/04eea015/2022-2023/Udinese-Stats', 'Sassuolo': 'https://fbref.com//en/squads/e2befd26/2022-2023/Sassuolo-Stats', 'Empoli': 'https

In [22]:
SerieA['Napoli']

'https://fbref.com//en/squads/d48ad4ff/2022-2023/Napoli-Stats'

In [48]:
cpt = 0
for i,j in SerieA.items():
    response = requests.get(j)
    print(response.status_code)

https://fbref.com//en/squads/d48ad4ff/2022-2023/Napoli-Stats
https://fbref.com//en/squads/7213da33/2022-2023/Lazio-Stats
https://fbref.com//en/squads/d609edc0/2022-2023/Internazionale-Stats
https://fbref.com//en/squads/dc56fe14/2022-2023/Milan-Stats
https://fbref.com//en/squads/922493f3/2022-2023/Atalanta-Stats
https://fbref.com//en/squads/cf74a709/2022-2023/Roma-Stats
https://fbref.com//en/squads/e0652b02/2022-2023/Juventus-Stats
https://fbref.com//en/squads/421387cf/2022-2023/Fiorentina-Stats
https://fbref.com//en/squads/1d8099f8/2022-2023/Bologna-Stats
https://fbref.com//en/squads/105360fe/2022-2023/Torino-Stats
https://fbref.com//en/squads/21680aa4/2022-2023/Monza-Stats
https://fbref.com//en/squads/04eea015/2022-2023/Udinese-Stats
https://fbref.com//en/squads/e2befd26/2022-2023/Sassuolo-Stats
https://fbref.com//en/squads/a3d88bd8/2022-2023/Empoli-Stats
https://fbref.com//en/squads/c5577084/2022-2023/Salernitana-Stats
https://fbref.com//en/squads/ffcbe334/2022-2023/Lecce-Stats
https

In [64]:
def Standard_st (table, club):
    if table:
        table_html = str(table[0])
        table_soup = BeautifulSoup(table_html, 'html.parser')

        player_rows = table_soup.find_all('tr')
        for row in player_rows:
            player_name = row.find('th', {'data-stat': 'player'}).text.strip()
            cl = club
            nationality = row.find('td', {'data-stat': 'nationality'}).text.strip()
            position = row.find('td', {'data-stat': 'position'}).text.strip()
            age = row.find('td', {'data-stat': 'age'}).text.strip()

            print(f"Player: {player_name}, Nationality: {nationality}, Position: {position}, Age: {age}, Club: {cl}")
        print()
        print()
    else:
        print("Table not found on the page.")


In [66]:
StandardStats = {}
advKeeper = {}
shooting = {}
passing = {}
passing_type = {}
gca = {}
defense = {}
possession = {}
playing_time = {}
misc = {}

z = 0
if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')
    for i,j in attr.items():
        table = soup.select(f'table#{j} tbody')
        print(z)
        z += 1
        Standard_st(table, 'Napoli')


0
Player: Tommaso Augello, Nationality: it ITA, Position: DF, Age: 27, Club: Napoli
Player: Tomás Rincón, Nationality: ve VEN, Position: MF, Age: 34, Club: Napoli
Player: Mehdi Léris, Nationality: dz ALG, Position: MF,DF, Age: 24, Club: Napoli
Player: Manolo Gabbiadini, Nationality: it ITA, Position: FW,MF, Age: 30, Club: Napoli
Player: Filip Đuričić, Nationality: rs SRB, Position: MF,FW, Age: 30, Club: Napoli
Player: Emil Audero, Nationality: it ITA, Position: GK, Age: 25, Club: Napoli
Player: Bruno Amione, Nationality: ar ARG, Position: DF, Age: 20, Club: Napoli
Player: Harry Winks, Nationality: eng ENG, Position: MF, Age: 26, Club: Napoli
Player: Bram Nuytinck, Nationality: nl NED, Position: DF, Age: 32, Club: Napoli
Player: Alessandro Zanoli, Nationality: it ITA, Position: DF, Age: 21, Club: Napoli
Player: Omar Colley, Nationality: gm GAM, Position: DF, Age: 29, Club: Napoli
Player: Bartosz Bereszyński, Nationality: pl POL, Position: DF, Age: 30, Club: Napoli
Player: Sam Lammers, N

In [ ]:
# Send a GET request to the webpage
url = 'https://fbref.com/en/squads/dc56fe14/2023-2024/all_comps/Milan-Stats-All-Competitions#all_stats_defense'
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Parse the HTML content
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Select the table containing player statistics
    table = soup.select('table#stats_standard_combined tbody')
    # Check if the table is found
    if table:
        # Convert the table to string and create a new BeautifulSoup object
        table_html = str(table[0])
        table_soup = BeautifulSoup(table_html, 'html.parser')
        
        # Find all table rows within the table
        player_rows = table_soup.find_all('tr')
        
        # Iterate over each row and extract player information
        for row in player_rows:
            # Extract player name
            player_name = row.find('th', {'data-stat': 'player'}).text.strip()
            
            # Extract other statistics
            nationality = row.find('td', {'data-stat': 'nationality'}).text.strip()
            position = row.find('td', {'data-stat': 'position'}).text.strip()
            age = row.find('td', {'data-stat': 'age'}).text.strip()
            games = row.find('td', {'data-stat': 'games'}).text.strip()
            goals = row.find('td', {'data-stat': 'goals'}).text.strip()
            assists = row.find('td', {'data-stat': 'assists'}).text.strip()
            
            # Print or store the extracted information
            print(f"Player: {player_name}, Nationality: {nationality}, Position: {position}, Age: {age}, Games: {games}, Goals: {goals}, Assists: {assists}")
    else:
        print("Table not found on the page.")
else:
    print("Failed to retrieve data. Status code:", response.status_code)

In [ ]:
import pandas as pd

# Assuming df is your DataFrame
# For demonstration purposes, let's create a sample DataFrame
data = {'Name': ['John', 'Emma', 'Ryan'],
        'Age': [25, 30, 35],
        'City': ['New York', 'Paris', 'London']}
df = pd.DataFrame(data)

# Export the DataFrame to a CSV file
df.to_csv('output.csv', index=False)
